In [9]:
using Revise
using HierarchicalDA
using TransportBasedInference
using LinearAlgebra
using Trixi
using OrdinaryDiffEq

In [2]:
gamma_gas = 1.4
equations = CompressibleEulerEquations1D(gamma_gas)

###############################################################################
# setup the GSBP DG discretization that uses the Gauss operators from 
# Chan, Del Rey Fernandez, Carpenter (2019). 
# [https://doi.org/10.1137/18M1209234](https://doi.org/10.1137/18M1209234)

# Shu-Osher initial condition for 1D compressible Euler equations
# Example 8 from Shu, Osher (1989).
# [https://doi.org/10.1016/0021-9991(89)90222-2](https://doi.org/10.1016/0021-9991(89)90222-2)
function initial_condition_shu_osher(x, t, equations::CompressibleEulerEquations1D)
    x0 = -4

    rho_left = 27 / 7
    v_left = 4 * sqrt(35) / 9
    p_left = 31 / 3

    # Replaced v_right = 0 to v_right = 0.1 to avoid positivity issues.
    v_right = 0.1
    p_right = 1.0

    rho = ifelse(x[1] > x0, 1 + 1 / 5 * sin(5 * x[1]), rho_left)
    v = ifelse(x[1] > x0, v_right, v_left)
    p = ifelse(x[1] > x0, p_right, p_left)

    return prim2cons(SVector(rho, v, p),
                     equations)
end

initial_condition = initial_condition_shu_osher

surface_flux = flux_lax_friedrichs
volume_flux = flux_ranocha

polydeg = 3
basis = DGMultiBasis(Line(), polydeg, approximation_type = GaussSBP())

indicator_sc = IndicatorHennemannGassner(equations, basis,
                                         alpha_max = 0.5,
                                         alpha_min = 0.001,
                                         alpha_smooth = true,
                                         variable = density_pressure)
volume_integral = VolumeIntegralShockCapturingHG(indicator_sc;
                                                 volume_flux_dg = volume_flux,
                                                 volume_flux_fv = surface_flux)

dg = DGMulti(basis,
             surface_integral = SurfaceIntegralWeakForm(surface_flux),
             volume_integral = volume_integral)

boundary_condition = BoundaryConditionDirichlet(initial_condition)
boundary_conditions = (; :entire_boundary => boundary_condition)

###############################################################################
#  setup the 1D mesh

cells_per_dimension = (64,)
mesh = DGMultiMesh(dg, cells_per_dimension,
                   coordinates_min = (-5.0,), coordinates_max = (5.0,),
                   periodicity = false)

###############################################################################
#  setup the semidiscretization and ODE problem

semi = SemidiscretizationHyperbolic(mesh, equations, initial_condition,
                                    dg, boundary_conditions = boundary_conditions)

tspan = (0.0, 1.0)
ode = semidiscretize(semi, tspan)

###############################################################################
#  setup the callbacks

# prints a summary of the simulation setup and resets the timers
summary_callback = SummaryCallback()

# analyse the solution in regular intervals and prints the results
analysis_callback = AnalysisCallback(semi, interval = 100, uEltype = real(dg))

# handles the re-calculation of the maximum Δt after each time step
stepsize_callback = StepsizeCallback(cfl = 0.1)

# collect all callbacks such that they can be passed to the ODE solver
callbacks = CallbackSet(summary_callback, analysis_callback, stepsize_callback)

# ###############################################################################
# # run the simulation

sol = solve(ode, SSPRK43(), adaptive = true, callback = callbacks, save_everystep = true)


████████╗██████╗ ██╗██╗  ██╗██╗
╚══██╔══╝██╔══██╗██║╚██╗██╔╝██║
   ██║   ██████╔╝██║ ╚███╔╝ ██║
   ██║   ██╔══██╗██║ ██╔██╗ ██║
   ██║   ██║  ██║██║██╔╝ ██╗██║
   ╚═╝   ╚═╝  ╚═╝╚═╝╚═╝  ╚═╝╚═╝

┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
│ SemidiscretizationHyperbolic                                                                     │
│ ════════════════════════════                                                                     │
│ #spatial dimensions: ………………………… 1                                                                │
│ mesh: ………………………………………………………………… Trixi.Affine DGMultiMesh with NDIMS = 1.                         │
│ equations: …………………………………………………… CompressibleEulerEquations1D                                     │
│ initial condition: ……………………………… initial_condition_shu_osher                                      │
│ boundary conditions: ………………………… 1                                                                

 L2 error:       2.92379549e-16   8.11781286e-16   3.00992649e-15
 Linf error:     1.33226763e-15   3.55271368e-15   1.42108547e-14
 ∑∂S/∂U ⋅ Uₜ :  -6.79117192e+01
────────────────────────────────────────────────────────────────────────────────────────────────────


────────────────────────────────────────────────────────────────────────────────────────────────────
 Simulation running 'CompressibleEulerEquations1D' with DGMulti(polydeg=3)
────────────────────────────────────────────────────────────────────────────────────────────────────
 #timesteps:                100                run time:       1.30741938e+00 s
 Δt:             1.72061138e-03                └── GC time:    1.16134580e-02 s (0.888%)
 sim. time:      1.32906843e-01 (13.291%)      time/DOF/rhs!:  2.45696104e-07 s
                                               PID:            1.05870143e-05 s
 #DOFs per field:           256                alloc'd memory:        184.858 MiB
 #elements:                  64

 Variable:  

retcode: Success
Interpolation: 3rd order Hermite
t: 606-element Vector{Float64}:
 0.0
 0.0006757636696956491
 0.0008746271150798364
 0.001414649350895998
 0.0018596348884632945
 0.0024560453631660268
 0.003186695786532508
 0.00400246359326881
 0.004882134048218675
 0.005896248816112489
 0.00710806534182598
 0.008482245032814823
 0.010042874208032643
 ⋮
 0.9780447804229789
 0.9796971697290797
 0.9812243093103029
 0.9827577265055162
 0.9841313895619335
 0.9858000192465726
 0.9878435831025992
 0.9903369034018803
 0.9931086545860628
 0.9959805491187541
 0.9990440463361037
 1.0
u: 606-element Vector{StructArray{SVector{3, Float64}, 2, Tuple{Matrix{Float64}, Matrix{Float64}, Matrix{Float64}}, Int64}}:
 [[3.8571428571428568, 10.1418510567422, 39.16666666666667] [3.8571428571428568, 10.1418510567422, 39.16666666666667] … [0.8015899452255287, 0.0801589945225529, 2.504007949726128] [0.8416355286787532, 0.08416355286787532, 2.5042081776433944]; [3.8571428571428577, 10.141851056742203, 39.1666666

In [3]:
x_vec = sol2vec(sol(0.1), equations)

768-element Vector{Float64}:
 3.8571238659256464
 3.857173549944555
 3.8571162646841755
 3.8572657526342424
 3.8573214262796904
 3.8570423302800374
 3.85725834225578
 3.856689864220949
 3.8565111173517503
 3.8574934502771834
 3.8567376187750786
 3.8585976687108112
 3.8591014541665523
 ⋮
 1.0000086718330359
 0.9999880511909189
 1.0000097362576572
 0.9999897969938482
 0.9999672916870809
 1.0000199854031513
 0.9999809400863566
 1.000009346347559
 0.9999672292321422
 1.0000050820747435
 1.0000225955929316
 0.9998376166213482

In [4]:
lines(vec(mesh.md.x), sol2vec(sol(0.1), equations))

LoadError: UndefVarError: `lines` not defined

In [31]:
Nvar = 3
Nxvar = 8
Nx = Nvar*Nxvar

24

In [32]:
X = randn(Nx, 200)

24×200 Matrix{Float64}:
 -1.31466     0.356333  -0.50798     …  -1.53207      1.30157   -1.35484
  2.53954     0.439815  -0.935759        0.117503     1.51859    0.997935
  0.426694    0.413553   0.44521         0.0519532   -0.796591  -0.10489
  0.726082    1.85953   -1.282          -1.68609     -0.264085  -1.82361
 -1.58613     0.11745    1.24096         1.25773     -1.66681   -0.407152
 -0.406011    1.36633   -0.840261    …  -1.05971     -1.04284   -0.0252338
  0.841749    0.450448  -0.219413       -0.76427     -0.839     -0.411332
 -0.995599    1.49733   -2.1398         -0.738122     0.261764   0.497849
 -0.322058   -1.53475    0.313376        0.569939    -1.05453   -0.88891
 -2.14058     0.307232  -1.04745         0.783867     0.865797  -1.21891
  0.234878   -0.984116   1.10496     …  -0.0497466   -0.809349   0.4424
  0.995861   -0.078886  -0.406298        2.00103      0.614724   0.207256
 -1.06478    -0.978703   1.52016        -0.244017    -0.516432  -2.13709
  1.41258    -1.22285

In [33]:
yidx = 1:1:Nx
idx = vcat(collect(1:length(yidx))', collect(yidx)')

# @assert length(yidx) == Ny

# # Create Localization structure
Gxx(i,j) = periodicmetric!(mod(i,Nxvar)-1,mod(j, Nxvar)-1, Nxvar)
Gxy(i,j) = periodicmetric!(i,yidx[j], Nxvar)
Gyy(i,j) = periodicmetric!(yidx[i],yidx[j], Nxvar)

Lrad = 1
Loc = Localization(Lrad, Gxx, Gxy, Gxx)

Localization(1.0, Gxx, Gxy, Gxx)

In [34]:
Ĉ = LocalizedEmpiricalCov(X, Loc)

LocalizedEmpiricalCov(24, 200, [-1.3146608020688086 0.35633320637200955 … 1.301567001314725 -1.3548446316745237; 2.5395400147991722 0.4398148735413339 … 1.5185926733563975 0.9979351511569552; … ; 0.42814884211492327 0.7804784267151214 … -0.7552816928422487 0.6981848374614592; 0.08438623715062298 0.37803501645141135 … 0.13129249643717475 -1.1304575612268162], [0.051131011795776685, 0.03201259396414227, -0.0059463241729616634, -0.10695400238313367, 0.18745695301809312, -0.08241365934018635, -0.04703980710568063, -0.10479299218290665, 0.011253809518454214, -0.09974515205335313  …  -0.03603919712401832, -0.06912539118399845, -0.09795926364612219, 0.02845028533374156, -0.048469810957437236, 0.012757213328944911, 0.03077548588084965, -0.07559495285205442, -0.1145164957669657, -0.11710910414875234], [1.2899142810297013 -0.04044280053268261 … -0.038954801491001625 0.07504754659497892; -0.04044280053268261 1.162372885184981 … 0.026920567879601915 -0.08433491802747234; … ; -0.038954801491001625 

In [38]:
Vector(Ĉ.ρX[1,:])

24-element Vector{Float64}:
 1.0
 0.20833333333333348
 1.6653345369377348e-16
 0.0
 0.0
 0.0
 1.6653345369377348e-16
 0.20833333333333348
 1.0
 0.20833333333333348
 1.6653345369377348e-16
 0.0
 0.0
 0.0
 1.6653345369377348e-16
 0.20833333333333348
 1.0
 0.20833333333333348
 1.6653345369377348e-16
 0.0
 0.0
 0.0
 1.6653345369377348e-16
 0.20833333333333348